In [1]:
from pymongo import MongoClient
import pyarabic as araby #module for normalizing arabic
from __future__ import absolute_import, print_function, unicode_literals, division
from pyarabic import araby as arabconst
import unicodedata
import re 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, TreebankWordTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.stem import ISRIStemmer

In [2]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)
# Access our database
db = client['arabic_database']
# Access our collection
collection = db['article']

articles = collection.find()

In [3]:
# Retrieve the first article
article = collection.find_one()
print(article)

{'_id': ObjectId('6610963b235e5906dd0df379'), 'url': 'https://althawrah.ye/archives/861481', 'title': 'مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية العام الحالي 2024م', 'pub_date': 'فبراير 13, 2024', 'category': 'اقتصاد', 'content': 'الثورة / متابعات أعلنت المفوضية السامية لشؤون اللاجئين التابعة للأمم المتحدة، عن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية المقدمة للنازحين واللاجئين في اليمن خلال العام 2024. وذكر تقرير صادر عن المفوضية أن عدد النازحين داخلياً في اليمن يقدر بنحو 4.56 مليون نازح، بالإضافة إلى أكثر من 70 ألف لاجئ وطالب لجوء. وبحسب التقرير فإن الأفراد والأسر النازحة معرضون للخطر بشكل خاص، مع انخفاض إمكانية الوصول إلى الخدمات الأساسية وسبل العيش، ويواجهون العديد من مخاطر الحماية غالباً، ويلجأ الكثيرون إلى آليات التكيف السلبية من أجل تدبر أمورهم، بما في ذلك تخطي وجبات الطعام، والتسرب من المدارس، وعمالة الأطفال، والحصول على القروض، والانتقال إلى مأوى أقل جودة، والزواج المبكر. وأوضحت المفوضية في تقريرها أن 85 % من المستفيدين من

In [4]:
# Extract only for now title and content
title = article['title']
content = article['content']
date = article['pub_date']

In [5]:
print(title)

مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية العام الحالي 2024م


In [6]:
print(content)

الثورة / متابعات أعلنت المفوضية السامية لشؤون اللاجئين التابعة للأمم المتحدة، عن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية المقدمة للنازحين واللاجئين في اليمن خلال العام 2024. وذكر تقرير صادر عن المفوضية أن عدد النازحين داخلياً في اليمن يقدر بنحو 4.56 مليون نازح، بالإضافة إلى أكثر من 70 ألف لاجئ وطالب لجوء. وبحسب التقرير فإن الأفراد والأسر النازحة معرضون للخطر بشكل خاص، مع انخفاض إمكانية الوصول إلى الخدمات الأساسية وسبل العيش، ويواجهون العديد من مخاطر الحماية غالباً، ويلجأ الكثيرون إلى آليات التكيف السلبية من أجل تدبر أمورهم، بما في ذلك تخطي وجبات الطعام، والتسرب من المدارس، وعمالة الأطفال، والحصول على القروض، والانتقال إلى مأوى أقل جودة، والزواج المبكر. وأوضحت المفوضية في تقريرها أن 85 % من المستفيدين من خدماتها يفضلون الحصول على الدعم نقداً (كلياً أو جزئيا)، مؤكدة أن المساعدات النقدية تمكن العائلات النازحة من شراء السلع والخدمات من الشركات المحلية، مما يعزز الاقتصاد المحلي، كما أنها واحدة من أكثر طرق الدعم سرعة وكفاءة وفعالية. وبلغ إجمالي المساعدات النقدية التي قدمتها المفو

In [7]:
print(date)

فبراير 13, 2024


# TEXT CLEANING

In [8]:
# Function to remove HTML tags
def remove_html_tags(text):
    html_pattern = re.compile(r'<[^>]+>')
    return html_pattern.sub('', text)

# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)
    
# Function to remove punctuation and special characters
def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text
    
# Function to remove Email IDs
def remove_email_ids(text):
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    return email_pattern.sub('', text)

# Function to remove Arabic diacritics (vowels and accents)
def remove_arabic_diacritics(text):
    arabic_diacritics_pattern = re.compile(r'[\u064B-\u065F\u0640]')
    return arabic_diacritics_pattern.sub('', text)
    
def remove_numbers(text):
    number_pattern = re.compile(r'\d+')
    return number_pattern.sub('', text)
    
# Main function to clean text
def clean_text(text):
    text = remove_html_tags(text)
    text = remove_urls(text)
    text = remove_punctuation(text)
    text = remove_email_ids(text)
    text = remove_arabic_diacritics(text)
    text = remove_numbers(text)
    return text

In [9]:
cleaned_title = remove_punctuation(title)
cleaned_content = remove_punctuation(content)

# Print results for the first article
cleaned_title = clean_text(title)
cleaned_content = clean_text(content)

print("\nOriginal Title:\n", title)
print("\nCleaned Title:\n", cleaned_title)
print("\nOriginal Content:\n", content)
print("\nCleaned Content:\n", cleaned_content)


Original Title:
 مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية العام الحالي 2024م

Cleaned Title:
 مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى  مليون دولار لتمويل المساعدات النقدية العام الحالي م

Original Content:
 الثورة / متابعات أعلنت المفوضية السامية لشؤون اللاجئين التابعة للأمم المتحدة، عن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية المقدمة للنازحين واللاجئين في اليمن خلال العام 2024. وذكر تقرير صادر عن المفوضية أن عدد النازحين داخلياً في اليمن يقدر بنحو 4.56 مليون نازح، بالإضافة إلى أكثر من 70 ألف لاجئ وطالب لجوء. وبحسب التقرير فإن الأفراد والأسر النازحة معرضون للخطر بشكل خاص، مع انخفاض إمكانية الوصول إلى الخدمات الأساسية وسبل العيش، ويواجهون العديد من مخاطر الحماية غالباً، ويلجأ الكثيرون إلى آليات التكيف السلبية من أجل تدبر أمورهم، بما في ذلك تخطي وجبات الطعام، والتسرب من المدارس، وعمالة الأطفال، والحصول على القروض، والانتقال إلى مأوى أقل جودة، والزواج المبكر. وأوضحت المفوضية في تقريرها أن 85 % من المستفيدين من خدماتها يفضلون 

In [10]:
# Define the functions for Arabic text normalization
def strip_tashkeel(text):
    """Strip vowel from a text and return a result text.
    The striped marks are :
        - FATHA, DAMMA, KASRA
        - SUKUN
        - SHADDA
        - FATHATAN, DAMMATAN, KASRATAN, , , .
    Example:
        >>> text=u"الْعَرَبِيّةُ"
        >>> strip_tashkeel(text)
        العربية

    @param text: arabic text.
    @type text: unicode.
    @return: return a striped text.
    @rtype: unicode.
    """
    return arabconst.strip_tashkeel(text)

In [11]:
#strip tatweel from a text and return a result text
def strip_tatweel(text):
    """
    Strip tatweel from a text and return a result text.

    Example:
        >>> text=u"العـــــربية"
        >>> strip_tatweel(text)
        العربية

    @param text: arabic text.
    @type text: unicode.
    @return: return a striped text.
    @rtype: unicode.
    """
    return arabconst.strip_tatweel(text)

In [12]:
def normalize_spellerrors(text):
    """Normalize some spellerrors like,
    TEH_MARBUTA into HEH,ALEF_MAKSURA into YEH, and return
    a result text.
    In some context users omit the difference between TEH_MARBUTA
    and HEH, and ALEF_MAKSURA and YEh.
    The conversions are:
        - TEH_MARBUTA into HEH
        - ALEF_MAKSURA into YEH

    Example:
        >>> text=u"اشترت سلمى دمية وحلوى"
        >>> normalize_spellerrors(text)
        اشترت سلمي دميه وحلوي

    @param text: arabic text.
    @type text: unicode.
    @return: return a converted text.
    @rtype: unicode.
    """
    text = re.sub(u'[%s]' % arabconst.TEH_MARBUTA, arabconst.HEH, text)
    return re.sub(u'[%s]' % arabconst.ALEF_MAKSURA, arabconst.YEH, text)

In [13]:
def normalize_lamalef(text):
    """Normalize Lam Alef ligatures into two letters (LAM and ALEF),
    and return a result text.
    Some systems present lamAlef ligature as a single letter,
    this function convert it into two letters,
    The converted letters into  LAM and ALEF are :
        - LAM_ALEF, LAM_ALEF_HAMZA_ABOVE, LAM_ALEF_HAMZA_BELOW,
         LAM_ALEF_MADDA_ABOVE

    Example:
        >>> text=u"لانها لالئ الاسلام"
        >>> normalize_lamalef(text)
        لانها لالئ الاسلام

    @param text: arabic text.
    @type text: unicode.
    @return: return a converted text.
    @rtype: unicode.
    """
    return arabconst.normalize_ligature(text)

In [14]:
def normalize_text(text):
    """Normalize input text and return a result text.
    Normalize a text by :
        - strip tashkeel
        - strip tatweel
        - normalize  Hamza
        - normalize Lam Alef.
        - normalize Teh Marbuta and Alef Maksura
    Example:
        >>> text=u'أستشتري دمـــى آلية لأبنائك قبل الإغلاق'
        >>> normalize_searchtext(text)
        استشتري دمي اليه لابناءك قبل الاغلاق

    @param text: arabic text.
    @type text: unicode.
    @return: return a normalized text.
    @rtype: unicode.
    """
    text = strip_tashkeel(text)
    text = strip_tatweel(text)
    text = normalize_lamalef(text)
    text = normalize_spellerrors(text)
    return text

In [15]:
normalized_title = normalize_text(cleaned_title)

print("\n Original Title:\n", cleaned_title)
print("\nNormalized Title:\n", normalized_title)


 Original Title:
 مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى  مليون دولار لتمويل المساعدات النقدية العام الحالي م

Normalized Title:
 مفوضيه شؤون اللاجئين في اليمن تعلن حاجتها إلي  مليون دولار لتمويل المساعدات النقديه العام الحالي م


In [16]:
normalized_content = normalize_text(cleaned_content)

print("\nOriginal Content:\n", cleaned_content)
print("\nNormalized Content:\n", normalized_content)


Original Content:
 الثورة  متابعات أعلنت المفوضية السامية لشؤون اللاجئين التابعة للأمم المتحدة عن حاجتها إلى  مليون دولار لتمويل المساعدات النقدية المقدمة للنازحين واللاجئين في اليمن خلال العام  وذكر تقرير صادر عن المفوضية أن عدد النازحين داخليا في اليمن يقدر بنحو  مليون نازح بالإضافة إلى أكثر من  ألف لاجئ وطالب لجوء وبحسب التقرير فإن الأفراد والأسر النازحة معرضون للخطر بشكل خاص مع انخفاض إمكانية الوصول إلى الخدمات الأساسية وسبل العيش ويواجهون العديد من مخاطر الحماية غالبا ويلجأ الكثيرون إلى آليات التكيف السلبية من أجل تدبر أمورهم بما في ذلك تخطي وجبات الطعام والتسرب من المدارس وعمالة الأطفال والحصول على القروض والانتقال إلى مأوى أقل جودة والزواج المبكر وأوضحت المفوضية في تقريرها أن   من المستفيدين من خدماتها يفضلون الحصول على الدعم نقدا كليا أو جزئيا مؤكدة أن المساعدات النقدية تمكن العائلات النازحة من شراء السلع والخدمات من الشركات المحلية مما يعزز الاقتصاد المحلي كما أنها واحدة من أكثر طرق الدعم سرعة وكفاءة وفعالية وبلغ إجمالي المساعدات النقدية التي قدمتها المفوضية خلال العام  نحو  

# TEXT PREPROCESSING

<div style="font-size:24px"> <p>We perform TOKENIZATION on original content and not the cleaned one first to see the differences between Sentence Tokenization and Word Tokenization (PunktWordTokenizer)</p></div>

In [17]:
# Sentence Tokenization
def tokenize_sentences(text):
    return sent_tokenize(text)

# Word Tokenization - using word_tokenize
def tokenize_words(text):
    return word_tokenize(text)

# Word Tokenization - using TreebankWordTokenizer
def tokenize_words_treebank(text):
    tokenizer = TreebankWordTokenizer()
    return tokenizer.tokenize(text)

# Word Tokenization - using WordPunctTokenizer
def tokenize_words_punkt(text):
    tokenizer = WordPunctTokenizer()
    return tokenizer.tokenize(text)

# Word Tokenization - using RegexpTokenizer
def tokenize_words_regexp(text):
    tokenizer = RegexpTokenizer("[\w']+")
    return tokenizer.tokenize(text)

In [18]:
print("Title Sentence Tokenization:")
print(tokenize_sentences(title))

print("\nTitle Word Tokenization - word_tokenize:")
print(tokenize_words(title))

print("\nTitle Word Tokenization - TreebankWordTokenizer:")
print(tokenize_words_treebank(title))

print("\nTitle Word Tokenization - WordPunctTokenizer:")
print(tokenize_words_punkt(title))

print("\nTitle Word Tokenization - RegexpTokenizer:")
print(tokenize_words_regexp(title))

Title Sentence Tokenization:
['مفوضية شؤون اللاجئين في اليمن تعلن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية العام الحالي 2024م']

Title Word Tokenization - word_tokenize:
['مفوضية', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلى', '100', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقدية', 'العام', 'الحالي', '2024م']

Title Word Tokenization - TreebankWordTokenizer:
['مفوضية', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلى', '100', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقدية', 'العام', 'الحالي', '2024م']

Title Word Tokenization - WordPunctTokenizer:
['مفوضية', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلى', '100', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقدية', 'العام', 'الحالي', '2024م']

Title Word Tokenization - RegexpTokenizer:
['مفوضية', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلى', '100', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقدية', 'العام', 'الحالي', '2024م']


In [19]:
print("\nContent Sentence Tokenization:")
print(tokenize_sentences(content))

print("\nContent Word Tokenization - word_tokenize:") 
print(tokenize_words(content))

print("\nContent Word Tokenization - TreebankWordTokenizer:")
print(tokenize_words_treebank(content))

print("\nContent Word Tokenization - WordPunctTokenizer:")
print(tokenize_words_punkt(content))

print("\nContent Word Tokenization - RegexpTokenizer:")
print(tokenize_words_regexp(content))


Content Sentence Tokenization:
['الثورة / متابعات أعلنت المفوضية السامية لشؤون اللاجئين التابعة للأمم المتحدة، عن حاجتها إلى 100 مليون دولار لتمويل المساعدات النقدية المقدمة للنازحين واللاجئين في اليمن خلال العام 2024.', 'وذكر تقرير صادر عن المفوضية أن عدد النازحين داخلياً في اليمن يقدر بنحو 4.56 مليون نازح، بالإضافة إلى أكثر من 70 ألف لاجئ وطالب لجوء.', 'وبحسب التقرير فإن الأفراد والأسر النازحة معرضون للخطر بشكل خاص، مع انخفاض إمكانية الوصول إلى الخدمات الأساسية وسبل العيش، ويواجهون العديد من مخاطر الحماية غالباً، ويلجأ الكثيرون إلى آليات التكيف السلبية من أجل تدبر أمورهم، بما في ذلك تخطي وجبات الطعام، والتسرب من المدارس، وعمالة الأطفال، والحصول على القروض، والانتقال إلى مأوى أقل جودة، والزواج المبكر.', 'وأوضحت المفوضية في تقريرها أن 85 % من المستفيدين من خدماتها يفضلون الحصول على الدعم نقداً (كلياً أو جزئيا)، مؤكدة أن المساعدات النقدية تمكن العائلات النازحة من شراء السلع والخدمات من الشركات المحلية، مما يعزز الاقتصاد المحلي، كما أنها واحدة من أكثر طرق الدعم سرعة وكفاءة وفعالية.', 'و

<div style="font-size:24px"> <p>Now we reperform Tokenization on the cleaned and normalized text, but there won´t be any difference because we removed the punctuation from our text</p></div> 

In [20]:
print("Title Sentence Tokenization:")
print(tokenize_sentences(normalized_title))

print("\nTitle Word Tokenization - word_tokenize:")
print(tokenize_words(normalized_title))

print("\nTitle Word Tokenization - TreebankWordTokenizer:")
print(tokenize_words_treebank(normalized_title))

print("\nTitle Word Tokenization - WordPunctTokenizer:")
print(tokenize_words_punkt(normalized_title))

print("\nTitle Word Tokenization - RegexpTokenizer:")
print(tokenize_words_regexp(normalized_title))

Title Sentence Tokenization:
['مفوضيه شؤون اللاجئين في اليمن تعلن حاجتها إلي  مليون دولار لتمويل المساعدات النقديه العام الحالي م']

Title Word Tokenization - word_tokenize:
['مفوضيه', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي', 'م']

Title Word Tokenization - TreebankWordTokenizer:
['مفوضيه', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي', 'م']

Title Word Tokenization - WordPunctTokenizer:
['مفوضيه', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي', 'م']

Title Word Tokenization - RegexpTokenizer:
['مفوضيه', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي', 'م']


In [21]:
print("\nContent Sentence Tokenization:")
print(tokenize_sentences(normalized_content))

print("\nContent Word Tokenization - word_tokenize:") 
print(tokenize_words(normalized_content))

print("\nContent Word Tokenization - TreebankWordTokenizer:")
print(tokenize_words_treebank(normalized_content))

print("\nContent Word Tokenization - WordPunctTokenizer:")
print(tokenize_words_punkt(normalized_content))

print("\nContent Word Tokenization - RegexpTokenizer:")
print(tokenize_words_regexp(normalized_content))


Content Sentence Tokenization:
['الثوره  متابعات أعلنت المفوضيه الساميه لشؤون اللاجئين التابعه للأمم المتحده عن حاجتها إلي  مليون دولار لتمويل المساعدات النقديه المقدمه للنازحين واللاجئين في اليمن خلال العام  وذكر تقرير صادر عن المفوضيه أن عدد النازحين داخليا في اليمن يقدر بنحو  مليون نازح بالإضافه إلي أكثر من  ألف لاجئ وطالب لجوء وبحسب التقرير فإن الأفراد والأسر النازحه معرضون للخطر بشكل خاص مع انخفاض إمكانيه الوصول إلي الخدمات الأساسيه وسبل العيش ويواجهون العديد من مخاطر الحمايه غالبا ويلجأ الكثيرون إلي آليات التكيف السلبيه من أجل تدبر أمورهم بما في ذلك تخطي وجبات الطعام والتسرب من المدارس وعماله الأطفال والحصول علي القروض والانتقال إلي مأوي أقل جوده والزواج المبكر وأوضحت المفوضيه في تقريرها أن   من المستفيدين من خدماتها يفضلون الحصول علي الدعم نقدا كليا أو جزئيا مؤكده أن المساعدات النقديه تمكن العائلات النازحه من شراء السلع والخدمات من الشركات المحليه مما يعزز الاقتصاد المحلي كما أنها واحده من أكثر طرق الدعم سرعه وكفاءه وفعاليه وبلغ إجمالي المساعدات النقديه التي قدمتها المفوضيه خلا

In [22]:
tokenized_title = tokenize_words(normalized_title)
tokenized_content = tokenize_words(normalized_content)

In [23]:
# Load Arabic stop words
stop_words = set(stopwords.words('arabic'))

In [24]:
# Tokenize the title and content
tokenizer = WordPunctTokenizer()
title_tokens = tokenizer.tokenize(normalized_title)
content_tokens = tokenizer.tokenize(normalized_content)

In [25]:
# Remove stop words from title
filtered_title = [w for w in title_tokens if not w in stop_words]
# Store stop words found
stop_words_found_title = [w for w in title_tokens if w in stop_words]


# Remove stop words from content
filtered_content = [w for w in content_tokens if not w in stop_words]
# Store stop words found
stop_words_found_content = [w for w in content_tokens if w in stop_words]

In [26]:
print("\nOriginal Title Tokens:", title_tokens)
print("\nFiltered Title Tokens:", filtered_title)
print("\nStop Words Found in Title:", stop_words_found_title)


print("\nOriginal Content Tokens:", content_tokens)
print("\nFiltered Content Tokens:", filtered_content)
print("\nStop Words Found in Content:", stop_words_found_content)



Original Title Tokens: ['مفوضيه', 'شؤون', 'اللاجئين', 'في', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي', 'م']

Filtered Title Tokens: ['مفوضيه', 'شؤون', 'اللاجئين', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي']

Stop Words Found in Title: ['في', 'دولار', 'م']

Original Content Tokens: ['الثوره', 'متابعات', 'أعلنت', 'المفوضيه', 'الساميه', 'لشؤون', 'اللاجئين', 'التابعه', 'للأمم', 'المتحده', 'عن', 'حاجتها', 'إلي', 'مليون', 'دولار', 'لتمويل', 'المساعدات', 'النقديه', 'المقدمه', 'للنازحين', 'واللاجئين', 'في', 'اليمن', 'خلال', 'العام', 'وذكر', 'تقرير', 'صادر', 'عن', 'المفوضيه', 'أن', 'عدد', 'النازحين', 'داخليا', 'في', 'اليمن', 'يقدر', 'بنحو', 'مليون', 'نازح', 'بالإضافه', 'إلي', 'أكثر', 'من', 'ألف', 'لاجئ', 'وطالب', 'لجوء', 'وبحسب', 'التقرير', 'فإن', 'الأفراد', 'والأسر', 'النازحه', 'معرضون', 'للخطر', 'بشكل', 'خاص', 'مع', 'انخفاض', 'إمكانيه', 'الوصول', 'إلي', 'الخدمات', 'الأساسيه', 'وس

In [27]:
stopwords.words('arabic')

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

# STEMMING vs LEMMATIZATION

## Stemming and Lemmatization in Natural Language Processing (NLP)

Stemming and lemmatization are techniques used to reduce words to their base form, but they differ in their approach and results.

### Stemming

* Removes suffixes from words to reach a stem.
* The stem may not be a valid word.
* Example: "running", "runners", and "run" become "run".
* Useful for tasks like search engines where the context is less important.

### Lemmatization

* Identifies the morphological root of a word based on context and part of speech.
* Produces a valid word, known as the lemma.
* Example: "am" becomes "be" and "running" becomes "run".
* Ideal for tasks requiring higher accuracy, such as machine translation and sentiment analysis.

## Arabic NLP: Stemming vs. Lemmatization Example

**Word:** "يكتبون" (yaktubūn) - "they write"

**Stemming:**

* Removes suffix "ون" (ūn), resulting in "يكتب" (yaktub).
* "يكتب" is not a valid word but carries the core meaning of "write".

**Lemmatization:**

* Analyzes morphology and context to find the base form: "كتب" (kataba) - "to write".
* "كتب" is a valid word and the dictionary form of the verb.

**Challenges in Arabic:**

* Rich morphology and vowel changes make accurate processing more complex.
* Dialectal variations further complicate the task.

**Arabic NLP Tools:**

* NLTK: Offers basic stemming.
* MADAMIRA: Provides comprehensive morphological analysis and lemmatization.
* Farasa: Offers both stemming and lemmatization functionalities.

**Choosing between Stemming and Lemmatization:**

* **Stemming:** Preferable for speed and simplicity in tasks like information retrieval.
* **Lemmatization:** Ideal for tasks requiring higher accuracy and understanding of word meaning.

<div style="font-size:24px"> <p>We perform stemming using ISRI lib for arabic stemming on the filtered text, & QALSADI for lemmetizing our normalized text:</p></div>

In [28]:
import qalsadi.lemmatizer

# Create an instance of the ISRIStemmer
stemmer = ISRIStemmer()

# Create an instance of the Lemmatizer class
lemmer = qalsadi.lemmatizer.Lemmatizer()

In [29]:
# Stem the title tokens
stemmed_title = [stemmer.stem(token) for token in filtered_title]

# Lemmatize title
title_lemmas = lemmer.lemmatize_text(normalized_title)

In [30]:
# Stem the content tokens
stemmed_content = [stemmer.stem(token) for token in filtered_content]

# Lemmatize content
content_lemmas = lemmer.lemmatize_text(normalized_content)

In [31]:
print("\nFiltered Title before Stemming: \n", filtered_title)
print("\nStemmed Title Tokens:\n", stemmed_title)

# Print the original title 
print("\nOriginal Title:")
print(normalized_title)

print("\nTitle Lemmas:\n", title_lemmas)


Filtered Title before Stemming: 
 ['مفوضيه', 'شؤون', 'اللاجئين', 'اليمن', 'تعلن', 'حاجتها', 'إلي', 'مليون', 'لتمويل', 'المساعدات', 'النقديه', 'العام', 'الحالي']

Stemmed Title Tokens:
 ['فوض', 'شؤن', 'لجئ', 'يمن', 'تعل', 'حجت', 'الي', 'ملي', 'مول', 'سعد', 'نقد', 'عام', 'الحالي']

Original Title:
مفوضيه شؤون اللاجئين في اليمن تعلن حاجتها إلي  مليون دولار لتمويل المساعدات النقديه العام الحالي م

Title Lemmas:
 ['مفوض', 'شؤون', 'لاجئ', 'في', 'يمن', 'علن', 'حاج', 'إلي', 'مليون', 'دولار', 'تمويل', 'مساعد', 'النقديه', 'عام', 'حال', 'م']


In [32]:
print("\nFiltered Content before Stemming:\n", filtered_content)
print("\nStemmed Content Tokens:\n", stemmed_content)

print("\nOriginal Content:")
print(normalized_content)

print("\nContent Lemmatized:\n", content_lemmas)


Filtered Content before Stemming:
 ['الثوره', 'متابعات', 'أعلنت', 'المفوضيه', 'الساميه', 'لشؤون', 'اللاجئين', 'التابعه', 'للأمم', 'المتحده', 'حاجتها', 'إلي', 'مليون', 'لتمويل', 'المساعدات', 'النقديه', 'المقدمه', 'للنازحين', 'واللاجئين', 'اليمن', 'خلال', 'العام', 'وذكر', 'تقرير', 'صادر', 'المفوضيه', 'عدد', 'النازحين', 'داخليا', 'اليمن', 'يقدر', 'بنحو', 'مليون', 'نازح', 'بالإضافه', 'إلي', 'لاجئ', 'وطالب', 'لجوء', 'وبحسب', 'التقرير', 'الأفراد', 'والأسر', 'النازحه', 'معرضون', 'للخطر', 'بشكل', 'خاص', 'انخفاض', 'إمكانيه', 'الوصول', 'إلي', 'الخدمات', 'الأساسيه', 'وسبل', 'العيش', 'ويواجهون', 'العديد', 'مخاطر', 'الحمايه', 'ويلجأ', 'الكثيرون', 'إلي', 'آليات', 'التكيف', 'السلبيه', 'تدبر', 'أمورهم', 'تخطي', 'وجبات', 'الطعام', 'والتسرب', 'المدارس', 'وعماله', 'الأطفال', 'والحصول', 'علي', 'القروض', 'والانتقال', 'إلي', 'مأوي', 'جوده', 'والزواج', 'المبكر', 'وأوضحت', 'المفوضيه', 'تقريرها', 'المستفيدين', 'خدماتها', 'يفضلون', 'الحصول', 'علي', 'الدعم', 'نقدا', 'كليا', 'جزئيا', 'مؤكده', 'المساعدات', 'النقد

## Comparison of Stemming and Lemmatization Outputs

| Aspect           | Stemming                                                    | Lemmatization                                                 |
| ---------------- | ----------------------------------------------------------- | -------------------------------------------------------------- |
| **Word Form**    | Reduced to stem, often not a valid word.                      | Reduced to lemma, always a valid dictionary form of the word. |
| **Meaning**      | May lose some nuances of meaning due to aggressive stemming. | Preserves meaning more accurately.                            |
| **Example**       | "تبع" (tba') - meaning unclear, not a valid word.             | "متابع" (mutabi') - "follower", a valid word with clear meaning. |
| **Readability**  | Less readable due to non-existent words.                   | More readable as it retains real words.                        |
| **Usefulness**   | Useful for tasks like keyword extraction and basic search.    | Ideal for tasks requiring understanding of context and meaning. |

## Conclusion

* **Stemming:** Simpler and faster, suitable for tasks prioritizing speed and basic word matching.
* **Lemmatization:** More sophisticated, preserves meaning, ideal for tasks requiring accurate understanding.

The choice depends on the specific NLP task and the desired balance between speed and accuracy.

## POS WITH NLTK

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Get English stop words
stop_words = set(stopwords.words('english'))

# Tokenize sentences
tokenized_sentences = sent_tokenize(normalized_content)

# Iterate over each sentence
for sentence in tokenized_sentences:
    # Tokenize words in the sentence
    wordsList = nltk.word_tokenize(sentence)
    # Remove stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words]
    # Perform part-of-speech tagging
    tagged = nltk.pos_tag(wordsList)
    print(tagged)

[('الثوره', 'JJ'), ('متابعات', 'NNP'), ('أعلنت', 'NNP'), ('المفوضيه', 'NNP'), ('الساميه', 'NNP'), ('لشؤون', 'NNP'), ('اللاجئين', 'NNP'), ('التابعه', 'NNP'), ('للأمم', 'NNP'), ('المتحده', 'NNP'), ('عن', 'NNP'), ('حاجتها', 'NNP'), ('إلي', 'NNP'), ('مليون', 'NNP'), ('دولار', 'NNP'), ('لتمويل', 'NNP'), ('المساعدات', 'NNP'), ('النقديه', 'NNP'), ('المقدمه', 'NNP'), ('للنازحين', 'NNP'), ('واللاجئين', 'NNP'), ('في', 'NNP'), ('اليمن', 'NNP'), ('خلال', 'NNP'), ('العام', 'NNP'), ('وذكر', 'NNP'), ('تقرير', 'NNP'), ('صادر', 'NNP'), ('عن', 'NNP'), ('المفوضيه', 'NNP'), ('أن', 'NNP'), ('عدد', 'NNP'), ('النازحين', 'NNP'), ('داخليا', 'NNP'), ('في', 'NNP'), ('اليمن', 'NNP'), ('يقدر', 'NNP'), ('بنحو', 'NNP'), ('مليون', 'NNP'), ('نازح', 'NNP'), ('بالإضافه', 'NNP'), ('إلي', 'NNP'), ('أكثر', 'NNP'), ('من', 'NNP'), ('ألف', 'NNP'), ('لاجئ', 'NNP'), ('وطالب', 'NNP'), ('لجوء', 'NNP'), ('وبحسب', 'NNP'), ('التقرير', 'NNP'), ('فإن', 'NNP'), ('الأفراد', 'NNP'), ('والأسر', 'NNP'), ('النازحه', 'NNP'), ('معرضون', 'NNP'

## NLTK's Limitations for Arabic POS Tagging

While NLTK provides basic POS tagging for Arabic, the output suggests potential inaccuracies and limitations:

**Observations:**

* Most words are tagged as nouns (NNP or NN), indicating potential misclassifications of verbs, adjectives, and other parts of speech.
* Arabic's complex morphology poses challenges for NLTK's rule-based approach.

**Limitations:**

* **Rule-Based Approach:** Insufficient for handling Arabic's intricacies. 
* **Limited Arabic Support:** Less developed compared to dedicated Arabic NLP tools.
* **Accuracy Concerns:** Output suggests potential inaccuracies.

**Recommendations:**

* **Explore other tools:** Consider CAMeL Tools (MADAMIRA) or Stanford CoreNLP for better accuracy.
* **Machine Learning Approaches:** Train a custom Arabic POS tagger with labeled data. 
* **Manual Verification:** Assess accuracy by comparing against manually annotated text.

**Overall, while NLTK offers a starting point, we will consider alternative solutions for tasks requiring high accuracy and comprehensive linguistic analysis of Arabic text.**

## With Stanford CoreNLP

In [34]:
from nltk.tag import StanfordPOSTagger

# Set the absolute paths to the Stanford POS Tagger jar file and model file
jar_path = '/home/sasamg/Downloads/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/stanford-postagger.jar'
model_path = '/home/sasamg/Downloads/stanford-tagger-4.2.0/stanford-postagger-full-2020-11-17/models/arabic.tagger'

tagger = StanfordPOSTagger(model_path, jar_path)

# Tag the tokens
tagged_tokens = tagger.tag(wordsList)

# Print the tagged tokens
print(tagged_tokens)

[('', 'الثوره/DTNN'), ('', 'متابعات/NNS'), ('', 'أعلنت/VBD'), ('', 'المفوضيه/DTNN'), ('', 'الساميه/DTJJ'), ('', 'لشؤون/NN'), ('', 'اللاجئين/DTNNS'), ('', 'التابعه/DTJJ'), ('', 'للأمم/NN'), ('', 'المتحده/DTNN'), ('', 'عن/IN'), ('', 'حاجتها/NN'), ('', 'إلي/CD'), ('', 'مليون/CD'), ('', 'دولار/NN'), ('', 'لتمويل/NNP'), ('', 'المساعدات/DTNNS'), ('', 'النقديه/DTJJ'), ('', 'المقدمه/DTJJ'), ('', 'للنازحين/NNS'), ('', 'واللاجئين/JJ'), ('', 'في/IN'), ('', 'اليمن/DTNNP'), ('', 'خلال/NN'), ('', 'العام/DTNN'), ('', 'وذكر/NN'), ('', 'تقرير/NN'), ('', 'صادر/JJ'), ('', 'عن/IN'), ('', 'المفوضيه/DTNN'), ('', 'أن/VBD'), ('', 'عدد/NN'), ('', 'النازحين/DTNNS'), ('', 'داخليا/JJ'), ('', 'في/IN'), ('', 'اليمن/DTNNP'), ('', 'يقدر/VBP'), ('', 'بنحو/NNP'), ('', 'مليون/CD'), ('', 'نازح/NN'), ('', 'بالإضافه/NNP'), ('', 'إلي/NNP'), ('', 'أكثر/NNP'), ('', 'من/IN'), ('', 'ألف/NN'), ('', 'لاجئ/NN'), ('', 'وطالب/NN'), ('', 'لجوء/NN'), ('', 'وبحسب/NN'), ('', 'التقرير/DTNN'), ('', 'فإن/NN'), ('', 'الأفراد/DTNN'), ('', 'و

## Stanford CoreNLP: A Robust Solution for Arabic POS Tagging

Analyzing the output, Stanford CoreNLP demonstrates strong capabilities:

**Observations:**

* **Diverse Tags:** Identifies various parts of speech, including determiners, nouns, verbs, adjectives, and prepositions, suggesting a more nuanced understanding of Arabic.
* **Morphological Analysis:** Provides morphological information alongside POS tags, indicating deeper linguistic analysis.
* **Potential Challenges:** Complex Arabic morphology might still present some challenges.

**Strengths:**

* **Accuracy:** Generally recognized for high accuracy in POS tagging for Arabic.
* **Sophistication:** Employs advanced linguistic models and algorithms for detailed analysis.
* **Feature Richness:** Offers a wide range of linguistic features beyond POS tags.

**Considerations:**

* **Resource Requirements:** Can be resource-intensive.
* **Setup Complexity:** Initial setup and configuration might be more involved.
* **Evaluation:** Accuracy should be assessed on specific data and domain.

**Overall, Stanford CoreNLP is a suitable choice for us.**

# NER WITH STANZA

In [40]:
import stanza
nlp = stanza.Pipeline('ar', processors='tokenize,ner') # initialize English neural pipeline
doc_title = nlp(cleaned_title) # run annotation over a sentence
for entity in doc_title.ents:
    print(f"Entity: {entity.text}\t Type: {entity.type}")

2024-04-08 01:47:58 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 01:47:58 INFO: Downloaded file to /home/sasamg/stanza_resources/resources.json
2024-04-08 01:47:58 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 01:47:59 INFO: Loading these models for language: ar (Arabic):
| Processor | Package      |
----------------------------
| tokenize  | padt         |
| mwt       | padt         |
| ner       | aqmar_charlm |

2024-04-08 01:47:59 INFO: Using device: cpu
2024-04-08 01:47:59 INFO: Loading: tokenize
2024-04-08 01:47:59 INFO: Loading: mwt
2024-04-08 01:47:59 INFO: Loading: ner
2024-04-08 01:47:59 INFO: Done loading processors!


Entity: اليمن	 Type: LOC


In [41]:
print(doc_title)
print(doc_title.entities)

[
  [
    {
      "id": 1,
      "text": "مفوضية",
      "start_char": 0,
      "end_char": 6,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "شؤون",
      "start_char": 7,
      "end_char": 11,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "اللاجئين",
      "start_char": 12,
      "end_char": 20,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "في",
      "start_char": 21,
      "end_char": 23,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 5,
      "text": "اليمن",
      "start_char": 24,
      "end_char": 29,
      "ner": "S-LOC",
      "multi_ner": [
        "S-LOC"
      ]
    },
    {
      "id": 6,
      "text": "تعلن",
      "start_char": 30,
      "end_char": 34,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": [
        7,
        8
      ],
    

In [42]:
nlp = stanza.Pipeline('ar', processors='tokenize,ner') # initialize English neural pipeline
doc_content = nlp(cleaned_content) # run annotation over a sentence
for entity in doc_content.ents:
    print(f"Entity: {entity.text}\t Type: {entity.type}")

2024-04-08 01:48:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-08 01:48:07 INFO: Downloaded file to /home/sasamg/stanza_resources/resources.json
2024-04-08 01:48:07 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 01:48:07 INFO: Loading these models for language: ar (Arabic):
| Processor | Package      |
----------------------------
| tokenize  | padt         |
| mwt       | padt         |
| ner       | aqmar_charlm |

2024-04-08 01:48:07 INFO: Using device: cpu
2024-04-08 01:48:07 INFO: Loading: tokenize
2024-04-08 01:48:07 INFO: Loading: mwt
2024-04-08 01:48:07 INFO: Loading: ner
2024-04-08 01:48:08 INFO: Done loading processors!


Entity: المفوضية السامية	 Type: ORG
Entity: لشؤون اللاجئين	 Type: ORG
Entity: للأمم المتحدة	 Type: ORG
Entity: واللاجئين	 Type: PER
Entity: اليمن	 Type: LOC
Entity: اليمن	 Type: LOC
Entity: دولار	 Type: MISC


In [43]:
print(doc_content)
print(doc_content.entities)

[
  [
    {
      "id": 1,
      "text": "الثورة",
      "start_char": 0,
      "end_char": 6,
      "ner": "O",
      "multi_ner": [
        "O"
      ],
      "misc": "SpacesAfter=\\s\\s"
    },
    {
      "id": 2,
      "text": "متابعات",
      "start_char": 8,
      "end_char": 15,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "أعلنت",
      "start_char": 16,
      "end_char": 21,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "المفوضية",
      "start_char": 22,
      "end_char": 30,
      "ner": "B-ORG",
      "multi_ner": [
        "B-ORG"
      ]
    },
    {
      "id": 5,
      "text": "السامية",
      "start_char": 31,
      "end_char": 38,
      "ner": "E-ORG",
      "multi_ner": [
        "E-ORG"
      ]
    },
    {
      "id": [
        6,
        7
      ],
      "text": "لشؤون",
      "start_char": 39,
      "end_char": 44,
      "ner": "B-ORG",
      "multi_ner